<a href="https://colab.research.google.com/github/nxxk23/PROJECT/blob/main/Test%5Bt%2B1%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
hemo = pd.read_csv('/content/drive/MyDrive/ckdresearch2023/Hemodialysis.csv', encoding='latin1')
hemo.shape

(149987, 5)

In [3]:
import pandas as pd
lab = pd.read_csv('/content/drive/MyDrive/ckdresearch2023/Lab.csv')
lab.shape

(21641523, 11)

In [4]:
patient_uid_list = hemo['PatientUID'].unique().tolist()
lab = lab[~lab['PatientUID'].isin(patient_uid_list)]
lab.shape

(17728574, 11)

In [5]:
result_item_names = ['eGFR(CKD-EPI) age >=18']
eGFR = lab[lab['ResultItemName'].isin(result_item_names)]
labeGFR = eGFR[['LabNo','PatientUID', 'PatientVisitUID', 'ResultItemName', 'ResultValue', 'CWhen']]
labeGFR

,LabNo,PatientUID,PatientVisitUID,ResultItemName,ResultValue,CWhen
248,1,2299,3386255,eGFR(CKD-EPI) age >=18,71.7,2559-08-15 09:51:54.913
255,1,2299,3640924,eGFR(CKD-EPI) age >=18,66.8,2559-11-11 09:40:06.053
271,1,2299,4020970,eGFR(CKD-EPI) age >=18,67.20,2560-03-31 10:18:54.363
286,1,2299,4286612,eGFR(CKD-EPI) age >=18,69.20,2560-07-05 10:40:40.763
331,1,2299,4546471,eGFR(CKD-EPI) age >=18,63.60,2560-10-03 09:19:54.710
...,...,...,...,...,...,...
21628049,159,925649,3617725,eGFR(CKD-EPI) age >=18,77.60,2560-04-17 08:54:35.740
21628297,160,925649,3617725,eGFR(CKD-EPI) age >=18,72.00,2560-04-19 08:41:45.060
21628538,161,925649,3617725,eGFR(CKD-EPI) age >=18,62.90,2560-04-21 08:47:03.167
21628775,162,925649,3617725,eGFR(CKD-EPI) age >=18,59.60,2560-04-24 08:59:53.310


In [6]:
eGFR = pd.pivot_table(labeGFR, index=['PatientUID', 'PatientVisitUID'], values=['ResultItemName', 'ResultValue', 'CWhen'], aggfunc='first')
eGFR

CWhen          ResultItemName  \
PatientUID PatientVisitUID                                                    
2299       3386255          2559-08-15 09:51:54.913  eGFR(CKD-EPI) age >=18   
           3640924          2559-11-11 09:40:06.053  eGFR(CKD-EPI) age >=18   
           4020970          2560-03-31 10:18:54.363  eGFR(CKD-EPI) age >=18   
           4286612          2560-07-05 10:40:40.763  eGFR(CKD-EPI) age >=18   
           4546471          2560-10-03 09:19:54.710  eGFR(CKD-EPI) age >=18   
...                                             ...                     ...   
3432056    10312896         2566-01-30 16:16:45.013  eGFR(CKD-EPI) age >=18   
           10408954         2566-02-24 21:05:12.747  eGFR(CKD-EPI) age >=18   
           10573233         2566-04-10 09:51:32.587  eGFR(CKD-EPI) age >=18   
3437526    10191122         2565-12-26 09:29:11.307  eGFR(CKD-EPI) age >=18   
           10541118         2566-03-30 18:44:40.160  eGFR(CKD-EPI) age >=18   

                           ResultValue  
PatientUID PatientVisitUID              
2299       3386255                71.7  
           3640924                66.8  
           4020970               67.20  
           4286612               69.20  
           4546471               63.60  
...                                ...  
3432056    10312896              72.86  
           10408954              68.69  
           10573233              45.08  
3437526    10191122              55.94  
           10541118              56.99  

[318686 rows x 3 columns]

In [7]:
import pandas as pd

df1 = pd.DataFrame(eGFR)

# Splitting the 'time' column into separate columns
df1[['date', 'timestamp']] = df1['CWhen'].str.split(' ', expand=True)
df1[['year', 'month', 'day']] = df1['date'].str.split('-', expand=True)
df1[['hour', 'minute', 'second']] = df1['timestamp'].str.split(':', expand=True)
df1[['second', 'nanosecond']] = df1['second'].str.split('.', expand=True)

# Convert year column from BE to AD
df1['year'] = df1['year'].astype(int) - 543

# Combine columns into a datetime column
df1['date'] = pd.to_datetime(df1[['year', 'month', 'day',]])
# Convert timestamp to Timedelta
df1['timestamp'] = pd.to_timedelta(df1['timestamp'])

# Convert date to datetime
df1['date'] = pd.to_datetime(df1['date'])

# Merge timestamp and date columns
df1['datetime'] = df1['date'] + df1['timestamp']

eGFR1 = df1.sort_values(by=['PatientUID', 'datetime'])

In [8]:
eGFR1 = eGFR1[['CWhen','ResultItemName','ResultValue','datetime']]
eGFR1_reset = eGFR1.reset_index()

In [10]:
import pandas as pd

validVisitID_list = []

for patientID, patient_df in eGFR1_reset.groupby('PatientUID'):
    validVisitID = [patient_df.iloc[0]['PatientVisitUID']]
    visitBefore = patient_df.iloc[0]['datetime']

    for _, row in patient_df.iloc[1:].iterrows():
        visitNow = row['datetime']

        if (visitNow - visitBefore).days >= 365:
            validVisitID.append(row['PatientVisitUID'])
            visitBefore = visitNow

    validVisitID_list.extend(validVisitID)

df_final = eGFR1_reset[eGFR1_reset['PatientVisitUID'].isin(validVisitID_list)]

In [11]:
df_final

,PatientUID,PatientVisitUID,CWhen,ResultItemName,ResultValue,datetime
0,2299,3386255,2559-08-15 09:51:54.913,eGFR(CKD-EPI) age >=18,71.7,2016-08-15 09:51:54.913
4,2299,4546471,2560-10-03 09:19:54.710,eGFR(CKD-EPI) age >=18,63.60,2017-10-03 09:19:54.710
10,2299,5812902,2561-12-12 09:18:49.913,eGFR(CKD-EPI) age >=18,70.20,2018-12-12 09:18:49.913
15,2299,6947220,2562-12-19 09:51:24.427,eGFR(CKD-EPI) age >=18,64.02,2019-12-19 09:51:24.427
23,2299,8186767,2564-03-04 10:01:42.113,eGFR(CKD-EPI) age >=18,48.54,2021-03-04 10:01:42.113
...,...,...,...,...,...,...
318672,3421578,10151787,2565-12-14 11:30:53.310,eGFR(CKD-EPI) age >=18,47.44,2022-12-14 11:30:53.310
318674,3426284,10105952,2565-11-29 10:47:37.917,eGFR(CKD-EPI) age >=18,27.88,2022-11-29 10:47:37.917
318676,3431501,10144649,2565-12-13 13:42:48.510,eGFR(CKD-EPI) age >=18,54.65,2022-12-13 13:42:48.510
318678,3432056,10154545,2565-12-15 09:48:31.117,eGFR(CKD-EPI) age >=18,70.18,2022-12-15 09:48:31.117


In [12]:
df_final['TimeDiff'] = df_final.groupby('PatientUID')['datetime'].diff().dt.days / 365
df_final['ResultValue'] = pd.to_numeric(df_final['ResultValue'], errors='coerce')
df_final['TimeDiff'] = pd.to_numeric(df_final['TimeDiff'], errors='coerce')
list_of_slopes = [(df_final['ResultValue'].iloc[i+1] - df_final['ResultValue'].iloc[i]) / df_final['TimeDiff'].iloc[i+1] for i in range(len(df_final)-1)]
df_final['Slopes'] = [None] + list_of_slopes

<ipython-input-12-d30c9da23213>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['TimeDiff'] = df_final.groupby('PatientUID')['datetime'].diff().dt.days / 365
<ipython-input-12-d30c9da23213>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['ResultValue'] = pd.to_numeric(df_final['ResultValue'], errors='coerce')
<ipython-input-12-d30c9da23213>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instea

In [13]:
df_final

,PatientUID,PatientVisitUID,CWhen,ResultItemName,ResultValue,datetime,TimeDiff,Slopes
0,2299,3386255,2559-08-15 09:51:54.913,eGFR(CKD-EPI) age >=18,71.70,2016-08-15 09:51:54.913,NaN,NaN
4,2299,4546471,2560-10-03 09:19:54.710,eGFR(CKD-EPI) age >=18,63.60,2017-10-03 09:19:54.710,1.131507,-7.158596
10,2299,5812902,2561-12-12 09:18:49.913,eGFR(CKD-EPI) age >=18,70.20,2018-12-12 09:18:49.913,1.189041,5.550691
15,2299,6947220,2562-12-19 09:51:24.427,eGFR(CKD-EPI) age >=18,64.02,2019-12-19 09:51:24.427,1.019178,-6.063710
23,2299,8186767,2564-03-04 10:01:42.113,eGFR(CKD-EPI) age >=18,48.54,2021-03-04 10:01:42.113,1.208219,-12.812245
...,...,...,...,...,...,...,...,...
318672,3421578,10151787,2565-12-14 11:30:53.310,eGFR(CKD-EPI) age >=18,47.44,2022-12-14 11:30:53.310,NaN,NaN
318674,3426284,10105952,2565-11-29 10:47:37.917,eGFR(CKD-EPI) age >=18,27.88,2022-11-29 10:47:37.917,NaN,NaN
318676,3431501,10144649,2565-12-13 13:42:48.510,eGFR(CKD-EPI) age >=18,54.65,2022-12-13 13:42:48.510,NaN,NaN
318678,3432056,10154545,2565-12-15 09:48:31.117,eGFR(CKD-EPI) age >=18,70.18,2022-12-15 09:48:31.117,NaN,NaN


In [15]:
data = pd.DataFrame(df_final)
data = data[['PatientUID','PatientVisitUID','ResultValue','datetime']]
data = data.sort_values(['PatientUID', 'PatientVisitUID'])
data['eGFR_t1'] = data.groupby('PatientUID')['ResultValue'].shift(-1)
data['time_t1'] = data.groupby('PatientUID')['datetime'].shift(-1)

In [17]:
data_t1 = data[['PatientUID','PatientVisitUID','eGFR_t1','time_t1']]
data_t1

,PatientUID,PatientVisitUID,eGFR_t1,time_t1
0,2299,3386255,63.60,2017-10-03 09:19:54.710
4,2299,4546471,70.20,2018-12-12 09:18:49.913
10,2299,5812902,64.02,2019-12-19 09:51:24.427
15,2299,6947220,48.54,2021-03-04 10:01:42.113
23,2299,8186767,80.47,2022-04-21 08:45:54.580
...,...,...,...,...
318672,3421578,10151787,NaN,NaT
318674,3426284,10105952,NaN,NaT
318676,3431501,10144649,NaN,NaT
318678,3432056,10154545,NaN,NaT


In [19]:
test = data_t1.dropna()
test

,PatientUID,PatientVisitUID,eGFR_t1,time_t1
0,2299,3386255,63.60,2017-10-03 09:19:54.710
4,2299,4546471,70.20,2018-12-12 09:18:49.913
10,2299,5812902,64.02,2019-12-19 09:51:24.427
15,2299,6947220,48.54,2021-03-04 10:01:42.113
23,2299,8186767,80.47,2022-04-21 08:45:54.580
...,...,...,...,...
317518,3319961,9270330,33.68,2023-04-04 08:28:58.833
317523,3319971,9270331,26.47,2023-04-04 08:49:20.047
317530,3320048,9242721,70.36,2023-04-10 08:36:15.217
317541,3320050,9257281,30.69,2023-04-10 08:58:57.267


In [20]:
test.to_excel('t1.xlsx')

In [14]:
slope = pd.read_excel('/content/drive/MyDrive/ckdresearch2023/Assign/SummaryFile/Slope.xlsx')
slope

,Unnamed: 0,PatientUID,PatientVisitUID,CWhen,ResultItemName,ResultValue,date,datetime,TimeDiff,Slopes
0,0,2299,3386255,2559-08-15 09:51:54.913,eGFR(CKD-EPI) age >=18,71.70,2016-08-15,2016-08-15 09:51:54.913,NaN,NaN
1,4,2299,4546471,2560-10-03 09:19:54.710,eGFR(CKD-EPI) age >=18,63.60,2017-10-03,2017-10-03 09:19:54.710,1.131507,-7.158596
2,10,2299,5812902,2561-12-12 09:18:49.913,eGFR(CKD-EPI) age >=18,70.20,2018-12-12,2018-12-12 09:18:49.913,1.189041,5.550691
3,15,2299,6947220,2562-12-19 09:51:24.427,eGFR(CKD-EPI) age >=18,64.02,2019-12-19,2019-12-19 09:51:24.427,1.019178,-6.063710
4,23,2299,8186767,2564-03-04 10:01:42.113,eGFR(CKD-EPI) age >=18,48.54,2021-03-04,2021-03-04 10:01:42.113,1.208219,-12.812245
...,...,...,...,...,...,...,...,...,...,...
73587,318672,3421578,10151787,2565-12-14 11:30:53.310,eGFR(CKD-EPI) age >=18,47.44,2022-12-14,2022-12-14 11:30:53.310,NaN,NaN
73588,318674,3426284,10105952,2565-11-29 10:47:37.917,eGFR(CKD-EPI) age >=18,27.88,2022-11-29,2022-11-29 10:47:37.917,NaN,NaN
73589,318676,3431501,10144649,2565-12-13 13:42:48.510,eGFR(CKD-EPI) age >=18,54.65,2022-12-13,2022-12-13 13:42:48.510,NaN,NaN
73590,318678,3432056,10154545,2565-12-15 09:48:31.117,eGFR(CKD-EPI) age >=18,70.18,2022-12-15,2022-12-15 09:48:31.117,NaN,NaN
